## Prep 1

In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle
import json



# from level2.level2_roi_extraction import level2_roi_extractor
# from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
# from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
# #import modules from files in a parallel directory "direct_regression"


# from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
# from direct_regression.get_all_series import get_all_subj_df
# from direct_regression.fmri_utils import *

import importlib


In [2]:
import modeling_utils
#importlib.reload(load_config)


In [3]:

config = modeling_utils.load_config("SST/direct_regression/config.yml")

glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-3.local
{'dev_scripts_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts', 'automotion_output_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/motion/motion_percent_summary_by_wave.csv', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'input_mat_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scriptsfMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analy

In [4]:
config

{'dev_scripts_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts',
 'automotion_output_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/motion/motion_percent_summary_by_wave.csv',
 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics',
 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'input_mat_path': '/Users/benjaminsmith/Documents/oregon/code/DEV_scriptsfMRI/fx/multiconds/SST/full_duration/conditions',
 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/',
 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/',
 'f

In [5]:
nonbids_data_path = config['nonbids_data_path']
dropbox_datapath = config['dropbox_data_dir']
automotion_datapath = config['automotion_output_path']
data_quality = pd.read_excel(dropbox_datapath + "/DEV-BothSessionsDataQualityC_DATA.xlsx", engine = 'openpyxl')

## SST

### W1 W2 analysis

In [ ]:
sst_data_missing_includes = pd.read_csv(dropbox_datapath + "/post-processing-fmri-data-inclusion.csv")
sst_data_missing_includes['SST'].unique()


In [ ]:
config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/"

In [ ]:
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
train_betas_with_data = modeling_utils.get_session_data_quality(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    task='SST'
    #subj_wave_inclusion='all'
)



In [ ]:
train_betas_with_data.to_csv(dropbox_datapath + "/beta_file_matches_sst.csv")

## Case study: SST data

Let's try to explain the size of the SST dataset.

In [ ]:
SST_selected_data = (
    train_betas_with_data[
        #redcap
        (train_betas_with_data.redcap_SST=='No reported problems')
        #motion exclude
        & (train_betas_with_data.motion_exclude_SST_Exclude.isna()==True)
        & (train_betas_with_data.motion_exclude_SST_Exclude != '')
         #betas
        & (train_betas_with_data.spm_output_path.isna()==False)
        & (train_betas_with_data.spm_output_path != '')
         #behavioral
        & (train_betas_with_data.data_by_ppt_merge_status=='participant_present')
    ]
)

In [ ]:
#now we separate by wave
SST_selected_data_w1 = SST_selected_data[SST_selected_data.wave_id==1]
#we have separate wave filters that also need to be applied.
SST_selected_data_w1.motion_exclude_SST_Exclude
SST_selected_data_w2 = SST_selected_data[SST_selected_data.wave_id==2]

In [ ]:
#now I guess we check to see which subjects exist in wave 1, wave 2, or both waves!
w1_subj_ids = SST_selected_data_w1.subject_id
w2_subj_ids = SST_selected_data_w2.subject_id
#now get the count of each, then the count of the intersection
w1_subj_ids.shape
w2_subj_ids.shape
#count of intersection
len(set(w1_subj_ids) & set(w2_subj_ids))

This may be a few more than we have had in the pipeline because I haven't used the now-obsolete exclusion list.

Now let's try again, getting that data, with a new function that will whittle down to just the subjects who have runs in each wave

In [ ]:

importlib.reload(modeling_utils)
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
subjects_in_all_waves = modeling_utils.get_sst_data_for_confirmed_sessions_across_tasks(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    subj_wave_inclusion='all'
)



In [ ]:
subjects_in_all_waves

# Prep 2

In [6]:
importlib.reload(modeling_utils)

<module 'modeling_utils' from '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py'>

In [7]:
session_quality_data = modeling_utils.get_overall_session_data_quality(
    dropbox_datapath=dropbox_datapath,
    automotion_datapath=automotion_datapath
    )

loaded 275 rows from data_by_ppt.csv
(543, 42) (570, 51) (571, 95)
(275, 174) (571, 95) (573, 268)
['SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'labelled_exclusion_wave', 'labelled_exclusion_missing wave', 'wave_id', 'automotion_exclude_wave']


/Users/benjaminsmith/anaconda/envs/neuroanalysis/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:726: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session.loc[:,col + '_quality'] = all_data_by_session[col].apply(lambda x: 1 if x=='No reported problems' else 0)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:726: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [8]:

importlib.reload(modeling_utils)
#get columns matching regex 'redcap_WTP\d_quality'
#and then create a list of the columns, then jsonify the list
#subjects_in_all_waves = modeling_utils.get_subject_wise_table_with_task_counts(all_sessions_wtp)
subjects_in_all_waves = modeling_utils.get_subject_wise_table_with_task_counts(session_quality_data)


# WTP

In [9]:
config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/"

'/Users/benjaminsmith/Documents/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/'

In [10]:
importlib.reload(modeling_utils)
#importlib.reload(level2.level2_utils)
#ultimately we can't make this completely task-agnostic, becasue we need to get a particular set of betas for each task
# all_sessions_wtp = modeling_utils.get_task_data_for_confirmed_sessions(
#     beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
#     #nonbids_data_path = config['nonbids_data_path'],
#     #ml_data_folderpath = ml_data_folderpath,
#     #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
#     dropbox_datapath=config['dropbox_data_dir'],
#     task='WTP'
# )



<module 'modeling_utils' from '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py'>

In [11]:
# #now save a tab delimited file with a list of subject IDs and json codes
# subjects_in_all_waves_wtp_include = ((subjects_in_all_waves.WTP_quality_list.isna()==False) & ((subjects_in_all_waves.WTP_quality_list.isin(['[[], []]','[[]]'])==False)))
# subjects_in_all_waves.loc[subjects_in_all_waves_wtp_include,['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality.csv", index=False,header=False, sep='\t')


In [12]:
#session_quality_data = modeling_utils.get_overall_session_data_quality(dropbox_datapath=dropbox_datapath, automotion_datapath = config['automotion_output_path'])
wtp_run_quality_data = modeling_utils.get_session_data_quality_l1(
    dropbox_datapath=dropbox_datapath,
    automotion_datapath=automotion_datapath,
    behavioral_data_folderpath = config['dev_scripts_path'] + "/fMRI/fx/multiconds/WTP/betaseries/",
    mastersheet_filepath=config['dropbox_data_dir'] + 'DEV Participant Mastersheet_copy.xlsx',
    task='WTP'
    )




/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:473: UserWarning: image_folder_glob is None; not checking for presence of images.
  warnings.warn("image_folder_glob is None; not checking for presence of images.")
/Users/benjaminsmith/anaconda/envs/neuroanalysis/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


loaded 275 rows from data_by_ppt.csv
(543, 42) (570, 51) (571, 95)
(275, 174) (571, 95) (573, 268)
(574, 285) (574, 285) (574, 285)


/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet_exclusion_data.rename(columns={'Dev ID#':'DEVID'}, inplace=True)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:726: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session.loc[:,col + '_quality'] = all_data_by_session[col].apply(lambda x: 1 if x=='No reported problems' else 0)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:

['SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'labelled_exclusion_wave', 'labelled_exclusion_missing wave', 'wave_id', 'automotion_exclude_wave', 'behav_present_subject_id', 'behav_present_wave_id']


/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:820: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session['combined_' + task + str(run_i) + '_quality' + str(run_i)] = (
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:828: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session[xrcn] = all_data_by_session[cols].apply(lambda x: ", ".join(x.index[x==0].tolist()),axis=1)
/Users/benjaminsmith/Documents/oregon/c

In [13]:
wtp_run_quality_data.to_csv(dropbox_datapath + "/wtp_run_data_quality_overall.csv", index=False)

Separate w1 and w2: 

In [16]:
# session_quality_data = modeling_utils.get_overall_session_data_quality(dropbox_datapath=dropbox_datapath, automotion_datapath = config['automotion_output_path'])
subject_wave_wise = modeling_utils.get_subject_wave_wise_table_with_task_counts(wtp_run_quality_data)
#now save a tab delimited file with a list of subject IDs and json codes
subjects_in_each_wave_wtp_include = ((subject_wave_wise.WTP_quality_list.isna()==False) & ((subject_wave_wise.WTP_quality_list.isin(['[[]]','[]'])==False)))
subject_wave_wise.loc[subjects_in_each_wave_wtp_include &(subject_wave_wise.wave_id==1) ,['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality_w1.tsv", index=False,header=False, sep='\t')
subject_wave_wise.loc[subjects_in_each_wave_wtp_include & (subject_wave_wise.wave_id==2),['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality_w2.tsv", index=False,header=False, sep='\t')


In [17]:
dropbox_datapath

'/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/'

In [ ]:
# subject_wave_wise = modeling_utils.get_subject_wave_wise_table_with_task_counts(wtp_run_quality_data)

# ROC

### Test code

In [18]:
importlib.reload(modeling_utils)

<module 'modeling_utils' from '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py'>

In [19]:
# session_quality_data = modeling_utils.get_overall_session_data_quality(dropbox_datapath=dropbox_datapath, automotion_datapath = config['automotion_output_path'])

In [20]:
# #now save a tab delimited file with a list of subject IDs and json codes
# subjects_in_each_wave_wtp_include = ((subject_wave_wise.ROC_quality_list.isna()==False) & ((subject_wave_wise.WTP_quality_list.isin(['[[]]'])==False)))
# subject_wave_wise.loc[subjects_in_each_wave_wtp_include & (subject_wave_wise.wave_id==1) ,['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality_w1.tsv", index=False,header=False, sep='\t')
# subject_wave_wise.loc[subjects_in_each_wave_wtp_include & (subject_wave_wise.wave_id==2),['SID','WTP_quality_list']].to_csv(dropbox_datapath + "/wtp_run_data_quality_w2.tsv", index=False,header=False, sep='\t')


### Main

In [21]:
# #now save a tab delimited file with a list of subject IDs and json codes
# subjects_in_all_waves_roc_include = ((subjects_in_all_waves.ROC_quality_list.isna()==False) & ((subjects_in_all_waves.ROC_quality_list.isin(['[[], []]','[[]]'])==False)))
# subjects_in_all_waves.loc[subjects_in_all_waves_roc_include,['SID','ROC_quality_list']].to_csv(dropbox_datapath + "/roc_run_data_quality.csv", index=False,header=False, sep='\t')


Save another ROC list that separates waves into separate rows, and saves files separately for wave 1 and wave 2

In [22]:
importlib.reload(modeling_utils)

<module 'modeling_utils' from '/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py'>

In [23]:
#session_quality_data = modeling_utils.get_overall_session_data_quality(dropbox_datapath=dropbox_datapath, automotion_datapath = config['automotion_output_path'])
roc_run_quality_data = modeling_utils.get_session_data_quality_l1(
    dropbox_datapath=dropbox_datapath,
    automotion_datapath=automotion_datapath,
    behavioral_data_folderpath = config['dev_scripts_path'] + "/fMRI/fx/multiconds/ROC/betaseries/",
    mastersheet_filepath=config['dropbox_data_dir'] + 'DEV Participant Mastersheet_copy.xlsx',
    task='ROC'
    )




/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:473: UserWarning: image_folder_glob is None; not checking for presence of images.
  warnings.warn("image_folder_glob is None; not checking for presence of images.")
/Users/benjaminsmith/anaconda/envs/neuroanalysis/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


loaded 275 rows from data_by_ppt.csv
(543, 42) (570, 51) (571, 95)
(275, 174) (571, 95) (573, 268)
(573, 285) (573, 285) (573, 285)


/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mastersheet_exclusion_data.rename(columns={'Dev ID#':'DEVID'}, inplace=True)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:726: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session.loc[:,col + '_quality'] = all_data_by_session[col].apply(lambda x: 1 if x=='No reported problems' else 0)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:

['SID', 'subject_id', 'redcap_dev_id', 'redcap_wave', 'labelled_exclusion_wave', 'labelled_exclusion_missing wave', 'wave_id', 'automotion_exclude_wave', 'behav_present_subject_id', 'behav_present_wave_id']


/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:828: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session[xrcn] = all_data_by_session[cols].apply(lambda x: ", ".join(x.index[x==0].tolist()),axis=1)
/Users/benjaminsmith/Documents/oregon/code/DEV_scripts/fMRI/fx/models/modeling_utils.py:820: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_by_session['combined_' + task + str(run_i) + '_quality' + str(run_i)] = (
/Users/benjaminsmith/Documents/oregon/c

In [24]:
roc_run_quality_data.to_csv(dropbox_datapath + "/roc_run_data_quality_overall.csv", index=False)

In [25]:
subject_wave_wise = modeling_utils.get_subject_wave_wise_table_with_task_counts(roc_run_quality_data)

In [26]:
#now save a tab delimited file with a list of subject IDs and json codes
subjects_in_each_wave_roc_include = ((subject_wave_wise.ROC_quality_list.isna()==False) & ((subject_wave_wise.ROC_quality_list.isin(['[[]]','[]'])==False)))
subject_wave_wise.loc[subjects_in_each_wave_roc_include & (subject_wave_wise.wave_id==1) ,['SID','ROC_quality_list']].to_csv(dropbox_datapath + "/roc_run_data_quality_w1.tsv", index=False,header=False, sep='\t')
subject_wave_wise.loc[subjects_in_each_wave_roc_include & (subject_wave_wise.wave_id==2),['SID','ROC_quality_list']].to_csv(dropbox_datapath + "/roc_run_data_quality_w2.tsv", index=False,header=False, sep='\t')


Now create files that store _excluded_ runs

In [ ]:
# subject_wave_wise.loc[subjects_in_each_wave_roc_include & (subject_wave_wise.wave_id==1), ['SID','ROC_quality_list']].to_csv(dropbox_datapath + "/roc_run_data_quality_w1.tsv", index=False,header=False, sep='\t')
# subject_wave_wise.loc[subjects_in_each_wave_roc_include & (subject_wave_wise.wave_id==2), ['SID','ROC_quality_list']].to_csv(dropbox_datapath + "/roc_run_data_quality_w2.tsv", index=False,header=False, sep='\t')


## Level 1 WTP

In [ ]:
importlib.reload(modeling_utils)
subjects_in_all_waves = modeling_utils.get_session_data_quality_l1(
    #image_folder_glob = config['bids_data_path'] + "fMRI/fx/models/WTP/all_waves/health_conditions/sub-DEV*/",
    #nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    #ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    subj_wave_inclusion='all',
    task='WTP'
)

# All data for archival purposes

In [ ]:

importlib.reload(modeling_utils)

session_quality_data = modeling_utils.get_overall_session_data_quality(dropbox_datapath=dropbox_datapath,automotion_datapath=automotion_datapath)

In [ ]:
session_quality_data.to_csv(dropbox_datapath + "/overall_data_quality.csv", index=False)